# **2021ML FALL HW1: PM2.5 Prediction (Regression)**

### Author: MLTAs

Methods:
* Training with all data
* Training (mini-batch=64, optimizer=Adam, learning rate=0.001)

Running this code on Google Colab probably passes simple baselines on Kaggle.

Tovpass


## **Download Data**


If the Google drive links are dead, you can download data from [kaggle](https://www.kaggle.com/c/ml2021spring-hw1/data), and upload data manually to the workspace.

In [ ]:
tr_path = 'train.csv'  # path to training data
tt_path = 'test.csv'   # path to testing data

!gdown --id '1AtgWdt7mXNBrai0ktxeGx3ZFMPu1B1KM' --output train.csv
!gdown --id '15MCDFuAW5Z4n2yXJg3VbyIAnP4Dz-Tn_' --output test.csv

# **Import Some Packages**

In [ ]:
# For data preprocess
import numpy as np
import csv
import math
import pandas as pd
import matplotlib.pyplot as plt

# **讀取training data**
讀入training data並對之進行preprocess

In [ ]:
def valid(x, y, mean):
# filter out extreme data
  for x1 in x:
    if (x1 >= mean*3):
      return False
  return y<mean*3

def parse2train(data, feats):
  x = []
  y = []

  pm25 = np.copy(data[17])
  mean = np.mean(pm25)

  total_length = data.shape[1] - 9
  for i in range(total_length):
    x_temp = data[0:feats, i:i+9]
    y_temp = pm25[i+9]
    
    if not valid(x_temp[17], y_temp, mean):
      continue
    
    x.append(x_temp.reshape(-1,))
    y.append(y_temp)
    
  x = np.array(x)
  y = np.array(y)
  return x,y

def normalize(x):
  for i in range(len(x)):
    for j in range(len(x[0])):
      if std[i] != 0:
        x[i][j] = (x[i][j] - mean[i]) / std[i]
  return x

#**This is our gradient descent algorithm. Adam was implemented.**

In [ ]:
def minibatch(x, y):
    batch_size = 512
    lr = 1e-3
    lam = 0.001
    beta_1 = np.full(x[0].shape, 0.9).reshape(-1, 1)  #shape:(9, 1)
    beta_2 = np.full(x[0].shape, 0.99).reshape(-1, 1)
    w = np.full(x[0].shape, 0.1).reshape(-1, 1)
    bias = 0.1
    m_t = np.full(x[0].shape, 0).reshape(-1, 1)
    v_t = np.full(x[0].shape, 0).reshape(-1, 1)
    m_t_b = 0.0
    v_t_b = 0.0
    t = 0
    epsilon = 1e-8
    print('begin y: ')
    print(y)
    for num in range(2000):
        index = np.arange(x.shape[0])
        np.random.shuffle(index)
        x = x[index]
        y = y[index]
        for b in range(int(x.shape[0]/batch_size)):
            t+=1
            x_batch = x[b*batch_size:(b+1)*batch_size]
            y_batch = y[b*batch_size:(b+1)*batch_size].reshape(-1,1)
            loss = y_batch - np.dot(x_batch,w) - bias

            g_t = np.dot(x_batch.transpose(),loss) * (-2) +  2 * lam * np.sum(w)
            g_t_b = loss.sum(axis=0) * (2)
            m_t = beta_1*m_t + (1-beta_1)*g_t 
            v_t = beta_2*v_t + (1-beta_2)*np.multiply(g_t, g_t)
            m_cap = m_t/(1-(beta_1**t))
            v_cap = v_t/(1-(beta_2**t))
            m_t_b = 0.9*m_t_b + (1-0.9)*g_t_b
            v_t_b = 0.99*v_t_b + (1-0.99)*(g_t_b*g_t_b) 
            m_cap_b = m_t_b/(1-(0.9**t))
            v_cap_b = v_t_b/(1-(0.99**t))
            w_0 = np.copy(w)
            
            w -= ((lr*m_cap)/(np.sqrt(v_cap)+epsilon)).reshape(-1, 1)
            bias -= (lr*m_cap_b)/(math.sqrt(v_cap_b)+epsilon)

            if num % 100 == 0:
              rmse = np.sqrt(np.mean(((np.dot(x, w) + bias) - y.reshape(-1,1)) ** 2))
              print(t, ':', str(rmse))
    
    print("final: ")
    print(np.dot(x, w) + bias)
    print("final y: ")
    print(y.reshape(-1,1))
    print("rmse")
    return w, bias

# **讀入testing data**
請注意testing的時候資料之間是沒有重疊的，如果要改寫這塊的話請務必注意

In [ ]:
feats = 18
data = pd.read_csv('train.csv')
for col in list(data.columns[2:]):
  data[col] = data[col].astype(str).map(lambda x: x.rstrip('x*#A'))
data = data.values
train_data = np.transpose(np.array(np.float64(data)))

mean = np.mean(train_data, axis = 1)
std = np.std(train_data, axis = 1)

train_x, train_y = parse2train(train_data, feats)

w, bias = minibatch(train_x, train_y)

print('w && bias:')
print(w)
print(bias)

begin y: 
[33. 41. 38. ... 14. 15. 13.]
1 : 481.7523345770117
2 : 476.6765714404782
3 : 471.60769408987954
4 : 466.54683226556944
5 : 461.4924694279128
6 : 456.45137699989124
7 : 451.4220837045181
8 : 446.4092116419954
9 : 441.3894772356395
10 : 436.4008383691054
11 : 431.41925148947996
12 : 426.44607414052615
13 : 421.4882390610703
14 : 416.52762372071004
15 : 411.58618004789093
16 : 406.6591759566988
17 : 401.7516380965219
18 : 396.8651984891848
19 : 391.9940875617033
20 : 387.1328336067474
21 : 382.28159164272694
22 : 377.45989181072923
23 : 372.67210189263943
24 : 367.91217785117874
25 : 363.1754836679797
26 : 358.4515397940557
27 : 353.7304613322393
28 : 349.03878070830854
29 : 344.3805170959404
30 : 339.73985583053644
31 : 335.1244584968868
32 : 330.53502552883583
3201 : 4.925005327026073
3202 : 4.955717613602948
3203 : 4.934864893324218
3204 : 4.937691466171677
3205 : 4.973254383214958
3206 : 4.938074244093292
3207 : 4.905905762508711
3208 : 4.964309157299148
3209 : 4.9649317536

In [ ]:
def parse2test(data, feats):
  x = []
  y = []

  # 用前面9筆資料預測下一筆PM2.5 所以需要-9
  total_length = data.shape[1] - 9
  for i in range(857):
    x_tmp = data[0:feats,9 * i: 9 * i + 9]
    x.append(x_tmp.reshape(-1,))
  # x 會是一個(n, 18, 9)的陣列， y 則是(n, 1) 
  x = np.array(x)
  return x


In [ ]:
data = pd.read_csv('test.csv')
data = data.values
test_data = np.transpose(np.array(np.float64(data)))
test_x = parse2test(test_data, feats)

# **預測testing set並寫成並寫成csv檔檔**

In [ ]:
 with open('my_sol.csv', 'w', newline='') as csvf:
    writer = csv.writer(csvf)
    writer.writerow(['Id','Predicted'])
    for i in range(int(test_x.shape[0])):
      writer.writerow([i + 1,(np.dot(np.reshape(w,-1),test_x[i]) + bias)[0]])